Ferramenta para desmontar bytecodes de contratos inteligentes em seus respectivos opcodes.

A ferramenta faz web scraping no site https://ethervm.io e coleta os bytecodes e seus respectivos opcodes, criando um dicionario que permite desmontar grandes quantidade de bytecodes (desde que estejam em um arquivo .csv).

Basta organizar o seu dataset com o cabeçalho 'bytecode' e 'category' e inserir ele em 'input_csv'. Ao rodar, será gerado um novo dataset chamado 'output.csv' que terá todos os opcodes respectivos ao seu dataset de origem inserido em 'input.csv'.

TO-DO: Aceitar outros tipos, como JSON.


In [20]:
import csv
import requests
from bs4 import BeautifulSoup

In [25]:
# web scraping
def fetch_bytecodes_and_opcodes(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Localize a tabela de opcodes
    table = soup.find('table', class_='opcodes') 
    rows = table.find_all('tr')

    # Dicionário
    bytecodes_to_opcodes = {}

    for row in rows:
        # tag <a> com os atributos id e name
        link = row.find('a', class_='anchor')
        if link and 'id' in link.attrs and 'name' in link.attrs:
            bytecode = link['id']
            opcode = link['name']

            bytecodes_to_opcodes[f"0x{bytecode}"] = opcode

    return bytecodes_to_opcodes

In [26]:
# Desmontador (disassembler)
def convert_bytecode_to_opcodes(bytecode, opcode_dict):
    if bytecode.startswith("0x"):
        bytecode = bytecode[2:]  # Remove o prefixo '0x'

    opcodes = []
    i = 0
    while i < len(bytecode):
        byte = bytecode[i:i+2].upper()  # Obter 2 caracteres
        opcode = opcode_dict.get(f"0x{byte}", f"UNKNOWN({byte})")

        # Ignorar opcodes desconhecidos
        if opcode.startswith("UNKNOWN"):
            i += 2  # Pular para o próximo byte
            continue

        # Simplificação de instruções variáveis
        # Estudar a necessidade das outras generalizações.
        if opcode.startswith("PUSH"):
            opcode = "PUSH"  # Generalizar todas as instruções PUSH
        elif opcode.startswith("DUP"):
            opcode = "DUP"  # Generalizar todas as instruções DUP
        elif opcode.startswith("SWAP"):
            opcode = "SWAP"  # Generalizar todas as instruções SWAP
        elif opcode.startswith("LOG"):
            opcode = "LOG"  # Generalizar todas as instruções LOG

        opcodes.append(opcode)

        # Tratar instruções PUSH (ignorar bytes adicionais)
        if opcode == "PUSH":
            push_bytes = int(byte, 16) - 0x60 + 1
            i += push_bytes * 2  # Ignorar os bytes PUSH
        i += 2  # Próximo byte

    return opcodes

In [27]:
# Ler, desmontar e criar um novo dataset
def process_csv(input_file, output_file, opcode_dict):
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        
        # Criando um novo arquivo CSV com as colunas 'Opcodes' e 'CATEGORY'
        with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
            fieldnames = ['Opcodes', 'CATEGORY']
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for row in reader:
                bytecode = row['bytecode']  # Use o nome exato da coluna de bytecode
                category = row['category']  # Use o nome exato da coluna de categoria
                opcodes = convert_bytecode_to_opcodes(bytecode, opcode_dict)
                opcodes_str = " ".join(opcodes) 

                writer.writerow({'Opcodes': opcodes_str, 'CATEGORY': category})

In [29]:
# Use a ferramenta aqui
url = "https://ethervm.io"  # URL onde os bytecodes e opcodes estão
bytecodes_opcodes = fetch_bytecodes_and_opcodes(url)

input_csv = 'final_dataset.csv'  # Nome do arquivo CSV de entrada
output_csv = 'output.csv'  # Nome do arquivo CSV de saída

process_csv(input_csv, output_csv, bytecodes_opcodes)